In [1]:
# Dependencies and Setup
import requests
import time
import pandas as pd
import numpy as np
from pprint import pprint
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.ticker as ticker
import sklearn.datasets as dta
import scipy.stats as st
from scipy.stats import sem
from scipy.stats import linregress
import seaborn as sns

In [3]:
base_url = 'https://api.opencovid.ca/summary?geo=pt&fill=true&version=true&pt_names=short&hr_names=hruid&fmt=json'
response = requests.get(base_url).json()
provinces = []
for province in response['data']:
    provinces.append(province['region'])
print(provinces) 

['AB', 'BC', 'MB', 'NB', 'NL', 'NS', 'NT', 'NU', 'ON', 'PE', 'QC', 'SK', 'YT']


In [5]:
# Set the base URL
covid_url = 'https://api.opencovid.ca/timeseries'
covid_confirmed_cases_response = requests.get(covid_url).json()
# Define an empty list to fetch the covid confirmed cases
pprint(covid_confirmed_cases_response['data']['vaccine_administration_total_doses']) 

[{'date': '2020-12-19',
  'name': 'vaccine_administration_total_doses',
  'region': 'AB',
  'value': 0,
  'value_daily': 0},
 {'date': '2020-12-20',
  'name': 'vaccine_administration_total_doses',
  'region': 'AB',
  'value': 0,
  'value_daily': 0},
 {'date': '2020-12-21',
  'name': 'vaccine_administration_total_doses',
  'region': 'AB',
  'value': 0,
  'value_daily': 0},
 {'date': '2020-12-22',
  'name': 'vaccine_administration_total_doses',
  'region': 'AB',
  'value': 0,
  'value_daily': 0},
 {'date': '2020-12-23',
  'name': 'vaccine_administration_total_doses',
  'region': 'AB',
  'value': 0,
  'value_daily': 0},
 {'date': '2020-12-24',
  'name': 'vaccine_administration_total_doses',
  'region': 'AB',
  'value': 0,
  'value_daily': 0},
 {'date': '2020-12-25',
  'name': 'vaccine_administration_total_doses',
  'region': 'AB',
  'value': 0,
  'value_daily': 0},
 {'date': '2020-12-26',
  'name': 'vaccine_administration_total_doses',
  'region': 'AB',
  'value': 4792,
  'value_daily': 4

  'region': 'NS',
  'value': 2610533,
  'value_daily': 0},
 {'date': '2023-02-23',
  'name': 'vaccine_administration_total_doses',
  'region': 'NS',
  'value': 2610533,
  'value_daily': 0},
 {'date': '2023-02-24',
  'name': 'vaccine_administration_total_doses',
  'region': 'NS',
  'value': 2610533,
  'value_daily': 0},
 {'date': '2023-02-25',
  'name': 'vaccine_administration_total_doses',
  'region': 'NS',
  'value': 2610533,
  'value_daily': 0},
 {'date': '2023-02-26',
  'name': 'vaccine_administration_total_doses',
  'region': 'NS',
  'value': 2618671,
  'value_daily': 8138},
 {'date': '2023-02-27',
  'name': 'vaccine_administration_total_doses',
  'region': 'NS',
  'value': 2618671,
  'value_daily': 0},
 {'date': '2023-02-28',
  'name': 'vaccine_administration_total_doses',
  'region': 'NS',
  'value': 2618671,
  'value_daily': 0},
 {'date': '2023-03-01',
  'name': 'vaccine_administration_total_doses',
  'region': 'NS',
  'value': 2618671,
  'value_daily': 0},
 {'date': '2023-03-02

 {'date': '2021-09-07',
  'name': 'vaccine_administration_total_doses',
  'region': 'YT',
  'value': 63316,
  'value_daily': 0},
 {'date': '2021-09-08',
  'name': 'vaccine_administration_total_doses',
  'region': 'YT',
  'value': 63316,
  'value_daily': 0},
 {'date': '2021-09-09',
  'name': 'vaccine_administration_total_doses',
  'region': 'YT',
  'value': 63316,
  'value_daily': 0},
 {'date': '2021-09-10',
  'name': 'vaccine_administration_total_doses',
  'region': 'YT',
  'value': 63316,
  'value_daily': 0},
 {'date': '2021-09-11',
  'name': 'vaccine_administration_total_doses',
  'region': 'YT',
  'value': 63564,
  'value_daily': 248},
 {'date': '2021-09-12',
  'name': 'vaccine_administration_total_doses',
  'region': 'YT',
  'value': 63564,
  'value_daily': 0},
 {'date': '2021-09-13',
  'name': 'vaccine_administration_total_doses',
  'region': 'YT',
  'value': 63564,
  'value_daily': 0},
 {'date': '2021-09-14',
  'name': 'vaccine_administration_total_doses',
  'region': 'YT',
  'va

In [6]:
# Set the base URL
covid_url = 'https://api.opencovid.ca/timeseries'
covid_vaccination_cases_response = requests.get(covid_url).json()
# Define an empty list to fetch the covid confirmed cases
covid_vaccination_cases_data = []
# Print to logger
print("Beginning Data Retrieval     ")
for case in covid_vaccination_cases_response['data']['vaccine_administration_total_doses']:
    covid_vaccination_cases_data.append({"Province": case['region'], "Date" : case['date'], "Cumulative vaccine administration total doses": case['value'], "Confirmed vaccine administration total doses per day": case['value_daily']})            
# Indicate that Data Loading is complete 
print("Data Retrieval Complete      ")

Beginning Data Retrieval     
Data Retrieval Complete      


In [8]:
covid_vaccination_cases_data_df = pd.DataFrame(covid_vaccination_cases_data)
# Show Record Count
covid_vaccination_cases_data_df.describe()

,Cumulative vaccine administration total doses,Confirmed vaccine administration total doses per day
count,1.090400e+04,1.090400e+04
mean,4.451261e+06,8.364963e+03
std,7.930348e+06,7.051809e+04
min,0.000000e+00,-1.913500e+05
25%,9.019200e+04,0.000000e+00
50%,1.320177e+06,0.000000e+00
75%,3.196579e+06,0.000000e+00
max,3.622686e+07,3.234393e+06


In [9]:
# Set the base URL
covid_url = 'https://api.opencovid.ca/timeseries'
covid_vaccinationdose1_coverage_response = requests.get(covid_url).json()
# Define an empty list to fetch the covid confirmed cases
covid_vaccinationdose1_coverage_data = []
# Print to logger
print("Beginning Data Retrieval     ")
for case in covid_vaccinationdose1_coverage_response['data']['vaccine_coverage_dose_1']:
    covid_vaccinationdose1_coverage_data.append({"Province": case['region'], "Date" : case['date'], "Cumulative vaccine_coverage_dose_1": case['value'], "Confirmed vaccine_coverage_dose_1 per day": case['value_daily']})            
# Indicate that Data Loading is complete 
print("Data Retrieval Complete      ")

Beginning Data Retrieval     
Data Retrieval Complete      


In [10]:
covid_vaccinationdose1_coverage_data_df = pd.DataFrame(covid_vaccinationdose1_coverage_data)
# Show Record Count
covid_vaccinationdose1_coverage_data_df

,Province,Date,Cumulative vaccine_coverage_dose_1,Confirmed vaccine_coverage_dose_1 per day
0,AB,2020-12-19,0.0,0.0
1,AB,2020-12-20,0.0,0.0
2,AB,2020-12-21,0.0,0.0
3,AB,2020-12-22,0.0,0.0
4,AB,2020-12-23,0.0,0.0
...,...,...,...,...
10899,YT,2023-04-19,84.8,0.0
10900,YT,2023-04-20,84.8,0.0
10901,YT,2023-04-21,84.8,0.0
10902,YT,2023-04-22,84.8,0.0
